In [1]:
## Libraries
import pandas as pd
import numpy as np
import scipy, scipy.stats

### 2. Write a function ‘exists’ which takes a variable symbol v and returns whether v is defined.

In [36]:
try:
    var
except NameError:
    print('v is not defined')
else:
    print("exists")

v is not defined


### 3. Create a function to compute N layer of a Pascal Triangle. 

In [37]:
def pascal_triangle(n):
    pascal = [[1]*(i+1) for i in range(n)] # set the initial layer
    
    # O(n^2)
    for i in range(n):
        for j in range(1,i):
            pascal[i][j] = pascal[i-1][j-1] + pascal[i-1][j]
    return pascal

In [38]:
# test cases for pascal triangle
pascal_triangle(10)
# pascal_triangle(20)

[[1],
 [1, 1],
 [1, 2, 1],
 [1, 3, 3, 1],
 [1, 4, 6, 4, 1],
 [1, 5, 10, 10, 5, 1],
 [1, 6, 15, 20, 15, 6, 1],
 [1, 7, 21, 35, 35, 21, 7, 1],
 [1, 8, 28, 56, 70, 56, 28, 8, 1],
 [1, 9, 36, 84, 126, 126, 84, 36, 9, 1]]

### 4. Assume have the following portfolio as of 2016/01/01:
    AAPL.O 15%
    IBM.N 20%
    GOOG.O 20%
    BP.N 15%
    XOM.N 10%
    COST.O 15%
    GS.N 5%

a. Using historical daily returns (Yahoo/Google Finance or any other market data
source), calculate VaR95% and CVaR95% of the portfolio as of 2016/12/31

b. Using expected mean, covariance matrix and parametric method, calculate VaR95%
and CVaR95%

c. Assume you can change weights, allow shorting but no leverage (i.e. sum of weights
equal 100%), and rebalance monthly. What is the optimal portfolio holding by end of
each month till end of 2016
Notes: If you have other assumption(s) please state clearly

#### a. Using historical daily returns (Yahoo/Google Finance or any other market data source), calculate VaR95% and CVaR95% of the portfolio as of 2016/12/31

In [34]:
portfolio = pd.read_csv('portfolio.csv')
portfolio['date'] = pd.to_datetime(portfolio['date'], format='%d/%m/%Y')

# calculate returns
returns = portfolio.set_index('date', inplace=True)
returns = portfolio.pct_change(1)
returns.dropna(axis=0, inplace=True)

# portfolio weights
weights = np.array([.15, .2, .2, .15, .1, .15, .05])

# covariance
cov = returns.cov()

# find the portfolio returns with weights
portfolio_returns = (weights * returns.values).sum(axis=1)

# portfolio std dev
portfolio_sd = np.sqrt(weights.T.dot(cov).dot(weights))

In [17]:
# VaR95% and CVaR95%
def historical_var(returns, alpha=5):
    return np.percentile(portfolio_returns, alpha)

def historical_cvar(returns, alpha=5):
    below_var = returns <= historical_var(returns, alpha=alpha)
    return returns[below_var].mean()

In [18]:
days = 1
investment = 100000
var95 = round(-historical_var(portfolio_returns, alpha=5)*np.sqrt(1)*investment, 2)
cvar95 = round(-historical_cvar(portfolio_returns, alpha=5)*np.sqrt(1)*investment, 2)

print('The VaR95% for a day with an investment of $100 000 is ${}'.format(var95))
print('The CVaR95% for a day with an investment of $100 000 is ${}'.format(cvar95))

The VaR95% for a day with an investment of $100 000 is $1425.18
The CVaR95% for a day with an investment of $100 000 is $2126.64


#### b. Using expected mean, covariance matrix and parametric method, calculate VaR95% and CVaR95%

In [24]:
# parametric functions for normal and t distribution

def var_parametric(portofolioReturns, portfolioStd, distribution='normal', alpha=5, dof=6):
    # because the distribution is symmetric
    if distribution == 'normal':
        VaR = scipy.stats.norm.ppf(1-alpha/100)*portfolioStd - portofolioReturns
    elif distribution == 't':
        # set degrees of freedom
        nu = dof
        VaR = np.sqrt((nu-2)/nu) * scipy.stats.t.ppf(1-alpha/100, nu) * portfolioStd - portofolioReturns
    return VaR

def cvar_parametric(portofolioReturns, portfolioStd, distribution='normal', alpha=5, dof=6):
    if distribution == 'normal':
        CVaR = (alpha/100)**-1 * scipy.stats.norm.pdf(scipy.stats.norm.ppf(alpha/100))*portfolioStd - portofolioReturns
    elif distribution == 't':
        # set degrees of freedom
        nu = dof
        xanu = scipy.stats.t.ppf(alpha/100, nu)
        CVaR = -1/(alpha/100) * (1-nu)**(-1) * (nu-2+xanu**2) * scipy.stats.t.pdf(xanu, nu) * portfolioStd - portofolioReturns
    return CVaR

In [23]:
var95_parametric_normal = round(var_parametric(portfolio_returns[-1], portfolio_sd)*np.sqrt(1)*investment, 2)
cvar95_parametric_normal = round(cvar_parametric(portfolio_returns[-1], portfolio_sd)*np.sqrt(1)*investment, 2)

var95_parametric_t = round(var_parametric(portfolio_returns[-1], portfolio_sd, distribution='t')*np.sqrt(1)*investment, 2)
cvar95_parametric_t = round(cvar_parametric(portfolio_returns[-1], portfolio_sd, distribution='t')*np.sqrt(1)*investment, 2)

In [21]:
print('The normal-distributed parametric VaR95% for a day with an investment of $100 000 is ${}'.format(var95_parametric_normal))
print('The normal-distributed parametric CVaR95% for a day with an investment of $100 000 is ${}'.format(cvar95_parametric_normal))

print('The t-distributed parametric VaR95% for a day with an investment of $100 000 is ${}'.format(var95_parametric_t))
print('The t-distributed parametric CVaR95% for a day with an investment of $100 000 is ${}'.format(cvar95_parametric_t))

The normal-distributed parametric VaR95% for a day with an investment of $100 000 is $2045.37
The normal-distributed parametric CVaR95% for a day with an investment of $100 000 is $2424.52
The t-distributed parametric VaR95% for a day with an investment of $100 000 is $1992.51
The t-distributed parametric CVaR95% for a day with an investment of $100 000 is $2509.32


#### c. Assume you can change weights, allow shorting but no leverage (i.e. sum of weights equal 100%), and rebalance monthly. What is the optimal portfolio holding by end of each month till end of 2016 Notes: If you have other assumption(s) please state clearly

In [42]:
## we use monthly rebalancing on portfolio weights based on sharpe ratio for optimisation

def calc_neg_sharpe(weights, mean_returns, cov, rf):
    portfolio_return = np.sum(mean_returns * weights) * 252
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(252)
    sharpe_ratio = (portfolio_return - rf) / portfolio_std
    return -sharpe_ratio

# constraints 
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

def max_sharpe_ratio(mean_returns, cov, rf):
    num_assets = len(mean_returns)
    args = (mean_returns, cov, rf)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))
    result = scipy.optimize.minimize(calc_neg_sharpe, num_assets*[1./num_assets,], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

In [40]:
# split the df into monthly
df = pd.read_csv('portfolio.csv')
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')
df.iloc[:, 1:8] = df.iloc[:, 1:8].pct_change()
df.dropna(axis=0, inplace=True)

# split into monthly
g = df.groupby(pd.Grouper(key='date', freq='M'))
# # groups to a list of dataframes with list comprehension
dfs = [group for _,group in g]

In [53]:
tickers = ['aapl', 'ibm', 'goog', 'bp', 'xom', 'cost', 'gs']
results = pd.DataFrame()

# iterate through each month to get the optimal weights
for month in dfs:
    mean_returns = month.iloc[:, 1:8].mean()
    cov = month.iloc[:, 1:8].cov()
    optimal_port_sharpe = max_sharpe_ratio(mean_returns, cov, 0.05)
    results = results.append(pd.DataFrame([round(x,2) for x in optimal_port_sharpe['x']],index=tickers).T)

In [54]:
results['month'] = ['jan', 'feb', 'march', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
results.set_index('month', inplace=True)
results

,aapl,ibm,goog,bp,xom,cost,gs
month,,,,,,,
jan,0.00,0.00,0.00,1.00,0.00,0.00,0.00
feb,0.00,0.75,0.00,0.00,0.25,0.00,0.00
march,0.16,0.53,0.00,0.00,0.00,0.30,0.00
apr,0.00,0.00,0.00,0.12,0.88,0.00,0.00
may,0.34,0.00,0.66,0.00,0.00,0.00,0.00
jun,0.00,0.00,0.00,0.18,0.00,0.82,0.00
jul,0.09,0.39,0.38,0.00,0.00,0.14,0.00
aug,0.00,0.00,0.00,0.00,0.00,0.00,1.00
sep,0.62,0.00,0.00,0.38,0.00,0.00,0.00
